In [1]:
import os, cv2

import numpy as np
import pandas as pd

from utils import crop

In [2]:
images = np.load("./Dataset/images.npy")
output_file = 'output_video.mp4'

In [3]:
FRAME_RATE = 30
COUNT, HEIGHT, WIDTH, CHANNEL = images.shape

In [4]:
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
video_writer = cv2.VideoWriter(output_file, fourcc, FRAME_RATE, (WIDTH, HEIGHT))

In [5]:
hog = cv2.HOGDescriptor()
hog.setSVMDetector(cv2.HOGDescriptor_getDefaultPeopleDetector())

def detect_human(image):
    (rects, weights) = hog.detectMultiScale(image, winStride=(4, 4))
    return rects

def get_all_people_bounds(image):
    all_rects = []
    rects = detect_human(image)
    all_rects.extend(rects)
    if len(rects) == 0:
        return []

    for (x, y, w, h) in rects:
        rect_img = crop(image, y, y+h, x, x+w)
        rects_img = get_all_people_bounds(rect_img)
        all_rects.extend(rects_img)

    return all_rects    

def get_people_count(image):
    out_image = image.copy()
    rects = get_all_people_bounds(image)
    count = len(rects)
    updated_rects = np.array([[x, y, x + w, y + h] for (x, y, w, h) in rects])
    for (xA, yA, xB, yB) in updated_rects:
        cv2.rectangle(out_image, (xA, yA), (xB, yB), (0, 255, 0), 2)

    cv2.putText(out_image, f"Total People in frame : {count}", (0, image.shape[0] - 20), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 0), 2)
    return count, out_image    

In [6]:
indices = []
people_counts = []

for index, image in enumerate(images):
    count, out_image = get_people_count(image)
    indices.append(index + 1)
    people_counts.append(count)

    video_writer.write(out_image)

video_writer.release()

In [ ]:
output_df = pd.DataFrame({"Indices": indices, "Count": people_counts})
output_df.head()

In [ ]:
output_df.to_csv("My_results.csv")